In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/test_sessions.csv
/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/train.zip
/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/site_dic.pkl
/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/sample_submission.csv
/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/train_sessions.csv


In [2]:
import numpy as np
import pandas as pd
import pickle

**Intro**

В рамках этого соревнования мы будем решать проблему обнаружения злоумышленника, анализируя его поведение. Это сложная и интересная проблема, сочетающая анализ данных и поведенческую психологию.

Например: Яндекс решает проблему обнаружения вторжений в почтовые ящики на основе моделей поведения пользователя. Короче говоря, модель поведения нарушителя может отличаться от модели владельца:

- нарушитель может не удалять электронные письма сразу после их прочтения, как это делает владелец почтового ящика;
- злоумышленник может помечать электронные письма и даже перемещать курсор по-другому
- и т.п.

Таким образом, злоумышленник может быть обнаружен и выброшен из почтового ящика. При этом владельцу может быть предложено пройти аутентификацию с помощью SMS-кода. Такой пилотный проект Яндекса описан в статье Хабре (https://habr.com/ru/company/yandex/blog/230583/).

Подобные вещи разрабатываются в Google Analytics и описываются в научных исследованиях. Вы можете найти более подробную информацию по этой теме, выполнив поиск по запросу «Traversal Pattern Mining» и «Sequential Pattern Mining».

В этом конкурсе мы собираемся решить аналогичную проблему: наш алгоритм должен проанализировать последовательность веб-сайтов, которые посещает конкретный пользователь, и спрогнозировать, является ли этот человек Алисой или это злоумышленником (тот,кто не является Алисов). В качестве метрики мы будем использовать метрику ROC AUC.

**Data Downloading and Transformation**

Загрузим тренировочный и тестовый наборы и проанализируем их.

In [3]:
train_df = pd.read_csv('/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/train_sessions.csv',
                      index_col='session_id')
test_df = pd.read_csv('/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/test_sessions.csv',
                     index_col='session_id')

In [4]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 21 columns):
site1     253561 non-null int64
time1     253561 non-null object
site2     250098 non-null float64
time2     250098 non-null object
site3     246919 non-null float64
time3     246919 non-null object
site4     244321 non-null float64
time4     244321 non-null object
site5     241829 non-null float64
time5     241829 non-null object
site6     239495 non-null float64
time6     239495 non-null object
site7     237297 non-null float64
time7     237297 non-null object
site8     235224 non-null float64
time8     235224 non-null object
site9     233084 non-null float64
time9     233084 non-null object
site10    231052 non-null float64
time10    231052 non-null object
target    253561 non-null int64
dtypes: float64(9), int64(2), object(10)
memory usage: 42.6+ MB


Набор обучающих данных содержит следующие атрибуты:

- **site1** – идентификатор первого посещенного сайта в сеансе
- **time1** – время посещения первого сайта в сеансе
- ...
- **site10** – идентификатор десятого посещенного сайта в сеансе
- **time10** – время посещения десятого сайта в сессии
- **target** – целевая переменная, имеет значение 1 для сеансов Алисы и 0 для сеансов других пользователей

Пользовательские сессии выбираются таким образом, чтобы они не превышали полчаса и/или содержали более десяти веб-сайтов. То есть сеанс считается завершенным, если пользователь посетил десять веб-сайтов или если сеанс длился более тридцати минут.

В таблице есть несколько пустых значений, это означает, что некоторые сеансы содержат менее десяти веб-сайтов. Замените пустые значения на 0 и измените типы столбцов на целочисленные значения. Также загрузите словарь веб-сайтов и проверьте, как он выглядит:

In [6]:
# Меняем тип атрибутов site1, ..., site10 на целочисленный и заменяем отсутствующие значения нулями
sites = ['site%s' % i for i in range(1,11)]
train_df[sites] = train_df[sites].fillna(0).astype(int)
test_df[sites] = test_df[sites].fillna(0).astype(int)

In [7]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,0,NaN,0,NaN,0,NaN,0,NaN,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
2,890,2014-02-22 11:19:50,941,2014-02-22 11:19:50,3847,2014-02-22 11:19:51,941,2014-02-22 11:19:51,942,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847,2014-02-22 11:19:52,3846,2014-02-22 11:19:52,1516,2014-02-22 11:20:15,1518,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39,2013-12-16 16:40:18,14768,2013-12-16 16:40:19,14769,2013-12-16 16:40:19,37,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768,2013-12-16 16:40:20,14768,2013-12-16 16:40:21,14768,2013-12-16 16:40:22,14768,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782,2014-03-28 10:52:42,782,2014-03-28 10:53:12,782,2014-03-28 10:53:42,782,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782,2014-03-28 10:55:12,782,2014-03-28 10:55:42,782,2014-03-28 10:56:12,782,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177,2014-02-28 10:55:22,175,2014-02-28 10:55:22,178,2014-02-28 10:55:23,177,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175,2014-02-28 10:55:59,177,2014-02-28 10:55:59,177,2014-02-28 10:57:06,178,2014-02-28 10:57:11,0


Вместе с набором данных поставляется словарь сайтов. Посмотрим его.

In [8]:
with open(r"/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

sites_dict = pd.DataFrame(list(site_dict.keys()), index=list(site_dict.values()), columns=['site'])

In [9]:
sites_dict.head()

,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


In [10]:
sites_dict.shape

(48371, 1)

In [11]:
train_df.shape, test_df.shape

((253561, 21), (82797, 20))

In [12]:
train_df['target'].values

array([0, 0, 0, ..., 0, 0, 0])

In [13]:
y_train = train_df['target'].values

Для самой базовой модели мы будем использовать только посещенные веб-сайты в сеансе (но мы не будем учитывать атрибуты меток времени). Смысл такого выбора: у Алисы есть свои любимые сайты, и чем чаще вы видите эти сайты в сеансе, тем выше вероятность того, что это сеанс Алисы, и наоборот.

Давайте подготовим данные, мы возьмем только атрибуты `site1, site2, ..., site10` из всего датасета. Имейте в виду, что пропущенные значения заменяются на ноль.

Преобразуем данные в формат, который можно передать в `CountVectorizer`.

In [14]:
idx = train_df.shape[0]
data = pd.concat([train_df, test_df], sort=False)

In [15]:
data[sites].to_csv('data_sessions_text.txt', 
                                 sep=' ', index=None, header=None)

In [16]:
!head data_sessions_text.txt

718 0 0 0 0 0 0 0 0 0
890 941 3847 941 942 3846 3847 3846 1516 1518
14769 39 14768 14769 37 39 14768 14768 14768 14768
782 782 782 782 782 782 782 782 782 782
22 177 175 178 177 178 175 177 177 178
570 21 570 21 21 0 0 0 0 0
803 23 5956 17513 37 21 803 17514 17514 17514
22 21 29 5041 14422 23 21 5041 14421 14421
668 940 942 941 941 942 940 23 21 22
3700 229 570 21 229 21 21 21 2336 2044


In [17]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(1, 1), max_features=50000)
with open('data_sessions_text.txt') as inp_file:
    data = cv.fit_transform(inp_file)

In [18]:
X_train = data[:idx]
X_test = data[idx:]
print(X_train.shape, X_test.shape)

(253561, 48362) (82797, 48362)


**Training the first model**

Итак, у нас есть алгоритм и данные для него. Давайте построим первую модель, используя реализацию логистической регрессии в Scikit-learn с параметрами по умолчанию. Для оценивания модели мы будем использовать отложенную выборка размера 1/10 от тренировочного набора.

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
log_reg = LogisticRegression(C=1.0, random_state=42, solver='lbfgs', max_iter=500)
X_train_log, X_valid_log, y_train_log, y_valid_log = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
log_reg.fit(X_train_log, y_train_log)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [20]:
y_pred = log_reg.predict_proba(X_valid_log)
score = roc_auc_score(y_valid_log, y_pred[:,1])
score

0.9613767578231046

Первая модель продемонстрировала качество 0,96. Давайте возьмем это значение в качестве отправной точки. Чтобы сделать прогноз на тестовом наборе, нужно снова обучить модель на всем тренировочном (до этого момента модель использовала только часть данных для обучения), что увеличит ее обобщающую способность.

In [21]:
log_reg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
# Делаем предсказания
y_test = log_reg.predict_proba(X_test)

In [23]:
y_test[:5]

array([[9.97690446e-01, 2.30955385e-03],
       [9.99999995e-01, 4.61401495e-09],
       [9.99999981e-01, 1.89667578e-08],
       [9.99999972e-01, 2.75422934e-08],
       [9.99967356e-01, 3.26442327e-05]])

In [24]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [25]:
write_to_submission_file(y_test[:,1], 'baseline_1.csv')

In [26]:
param_grid = [
    {'penalty' : ['l1', 'l2'],
    'C' : [0.001,0.01,0.1,1,10,100,1000],
    'solver' : ['liblinear']}]

In [27]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(log_reg, scoring = 'roc_auc', param_grid = param_grid, cv = 5, verbose = True, n_jobs = -1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   52.8s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:  6.6min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=500, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=42, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                          'penalty': ['l1', 'l2'], 'solver': ['liblinear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=True)

In [28]:
grid_search.best_params_

{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}

In [29]:
y_test = grid_search.best_estimator_.predict_proba(X_test)

In [30]:
y_test

array([[9.97570132e-01, 2.42986817e-03],
       [9.99999995e-01, 5.10599228e-09],
       [9.99999981e-01, 1.88419076e-08],
       ...,
       [9.91156738e-01, 8.84326233e-03],
       [9.99525444e-01, 4.74556322e-04],
       [9.99977370e-01, 2.26304794e-05]])

In [31]:
write_to_submission_file(y_test[:,1], 'baseline_2.csv')